## Words and Tokens

In [5]:
#simple rule based tokenization
import re
def simple_tokenize(text):
    text = text.lower()
    tokens = re.findall(r"[a-z]+(?:'[a-z]+)?",text)
    #[a-z]+ matches the main word part (e.g. "don").
    #(?:'[a-z]+) optionally matches 't part (apostrophe + letters).
    # don't , we're, it's
    return tokens

In [6]:
text = "hello how are you, I'm fine, I hope this letter finds you in pink of health"
tokens = simple_tokenize(text)
print(tokens)

['hello', 'how', 'are', 'you', "i'm", 'fine', 'i', 'hope', 'this', 'letter', 'finds', 'you', 'in', 'pink', 'of', 'health']


In [7]:
#BPE is the binary pair encoder, were we divide words into subwords or small set of letters which often appear together.
from collections import Counter, defaultdict 

def bpe_train(corpus_tokens, vocab_size = 1000):
    
    words = [' '.join(list(w)) + '</w>' for w in corpus_tokens]
    word_counts = Counter(words)
    def get_stats(word_counts):
        pairs = Counter()
        for w,c in word_counts.items():
            symbols = w.split()
            for i in range(len(symbols)-1):
                pairs[(symbols[i], symbols[i+1])] += c
        return pairs

    merges = []
    while True:
        stats = get_stats(word_counts)
        if not stats: break
        (a,b) ,freq = stats.most_common(1)[0]
        if len(merges) + 256 >= vocab_size: break
        merges.append((a,b))
        new_counts = Counter()
        bigram = " ".join((a,b))
        for w,c in word_counts.items():
            new_w = w.replace(bigram, a+b)
            new_counts[new_w] += c
        word_counts = new_counts

    return merges

def bpe_encode(word, merges):
    w = list(word) + ['</w>']
    merges_set = {tuple(m) for m in merges}
    while True:
        pairs = [(i,(w[i], w[i+1])) for i in range(len(w)-1)]
        idx= next((i for i,p in pairs if p in merges_set), None)
        if idx is None: break 
        w = w[:idx] + [w[idx] + w[idx+1]] + w[idx+2:]
    return [t for t in w if t != '</w>']
        

In [9]:
from PyPDF2 import PdfReader
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path,'rb') as f:
        reader = PdfReader(f)
        for page_num, page in enumerate(reader.pages):
            text += page.extract_text() or ""
    return text 

pdf_path = "ed3book_aug25 no changes.pdf"
corpus_text = extract_text_from_pdf(pdf_path)

print("sample text: \n", corpus_text[:500])

sample text: 
 Speech and Language Processing
An Introduction to Natural Language Processing,
Computational Linguistics, and Speech Recognition
with Language Models
Third Edition draft
Daniel Jurafsky
Stanford University
James H. Martin
University of Colorado at Boulder
Copyright ©2025. All rights reserved.
Draft of August 24, 2025. Comments and typos welcome!Summary of Contents
I Large Language Models 1
1 Introduction. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .


In [11]:
tokens = simple_tokenize(corpus_text)
tokens_merges = bpe_train(tokens, vocab_size = 20000)

In [21]:
bpe_encode('language', tokens_merges)

['la', 'ngu', 'ag', 'e']

In [19]:
tokens_merges[:10]

[('t', 'h'),
 ('i', 'n'),
 ('t', 'i'),
 ('th', 'e</w>'),
 ('e', 'n'),
 ('a', 'n'),
 ('r', 'e'),
 ('c', 'o'),
 ('t', 'e'),
 ('o', 'n</w>')]

### Edit Distance (Levenshtein) 
This is a dynamic programming concept where we create a table dynamically and find the shortest path or efficient edit distance between two words. 

In [16]:
def edit_distance(a,b):
    n,m = len(a), len(b)
    dp = [[0]*(m+1) for _ in range(n+1)]
    for i in range(n+1): dp[i][0] = i
    for j in range(m+1): dp[0][j] = j
    for i in range(1,n+1):
        for j in range(1,m+1):
            cost = 0 if a[i-1]==b[j-1] else 1
            dp[i][j] = min(dp[i-1][j]+1, dp[i][j-1]+1, dp[i-1][j-1]+cost)
    return dp[n][m]

In [18]:
edit_distance('model','modal')

1